Тема: Основы статистики и статистические критерии

Видео лекции:  
https://www.youtube.com/watch?v=v4ydaa-94nI
    
Видео семинара:  
https://www.youtube.com/watch?v=VKuY4ApfH6M

# Домашнее задание к 1-му уроку

## Задача 1. Оценка вероятности ошибки I рода.
Напишите функцию для оценки ошибки первого рода.

In [1]:
import numpy as np
import pandas as pd
from scipy.stats import ttest_ind


def estimate_first_type_error(df_pilot_group, df_control_group, metric_name, alpha=0.05, n_iter=10000, seed=None):
    """Оцениваем ошибку первого рода.

    Бутстрепим выборки из пилотной и контрольной групп тех же размеров, считаем долю случаев с значимыми отличиями.
    
    df_pilot_group - pd.DataFrame, датафрейм с данными пилотной группы
    df_control_group - pd.DataFrame, датафрейм с данными контрольной группы
    metric_name - str, названия столбца с метрикой
    alpha - float, уровень значимости для статтеста
    n_iter - int, кол-во итераций бутстрапа
    seed - int or None, состояние генератора случайных чисел.

    return - float, ошибка первого рода
    """
    p_vals = []
    for _ in range(n_iter):
        a = df_control_group[metric_name].sample(frac=1, replace=True, random_state=seed)
        b = df_pilot_group[metric_name].sample(frac=1, replace=True, random_state=seed)
        _, p_val = ttest_ind(a, b)
        p_vals.append(p_val)
        
    res = sum([1 if p_val < alpha else 0 for p_val in p_vals]) / n_iter
    return res      

In [3]:
df_1 = pd.DataFrame({'vals': np.random.normal(0, 1, size=1000)})
df_2 = pd.DataFrame({'vals': np.random.normal(0, 1, size=1000)})

estimate_first_type_error(df_1, df_2, 'vals')

0.0959

## Задача 2. Оценка вероятности ошибки II рода.

Напишите функцию для оценки ошибки второго рода.

Добавлять эффект к экспериментальной группе можно разными способами. Способ добавления эффекта может повлиять на результаты статистических критериев и оценку вероятности ошибки II рода. При выполнении задания считайте, что **эффект распределён равномерно** по всем объектам экспериментальной группы, то есть метрики объектов в среднем увеличиваются на **одинаковую величину**.

In [16]:
import numpy as np
import pandas as pd
from scipy.stats import ttest_ind


def estimate_second_type_error(df_pilot_group, df_control_group, metric_name, effects, alpha=0.05, n_iter=10000, seed=None):
    """Оцениваем ошибки второго рода.

    Бутстрепим выборки из пилотной и контрольной групп тех же размеров, добавляем эффект к пилотной группе,
    считаем долю случаев без значимых отличий.
    
    df_pilot_group - pd.DataFrame, датафрейм с данными пилотной группы
    df_control_group - pd.DataFrame, датафрейм с данными контрольной группы
    metric_name - str, названия столбца с метрикой
    effects - List[float], список размеров эффектов ([1.03] - увеличение на 3%).
    alpha - float, уровень значимости для статтеста
    n_iter - int, кол-во итераций бутстрапа
    seed - int or None, состояние генератора случайных чисел

    return - dict, {размер_эффекта: ошибка_второго_рода}
    """
    dict_res = dict()
    for effect in effects:
        p_vals = []
        for _ in range(n_iter):
            a = df_control_group[metric_name].sample(frac=1, replace=True, random_state=seed)
            b = df_pilot_group[metric_name].sample(frac=1, replace=True, random_state=seed)
            b *= effect
            _, p_val = ttest_ind(a, b)
            p_vals.append(p_val)

        res = sum([1 if p_val >= alpha else 0 for p_val in p_vals]) / n_iter
        dict_res[effect] = res
        
    return dict_res  

In [17]:
SIZE = 1000
df_1 = pd.DataFrame({'vals': np.random.normal(loc=1, scale=1, size=SIZE)})
df_2 = pd.DataFrame({'vals': np.random.normal(loc=1, scale=1, size=SIZE)})
effects = [1.03, 1.05, 1.10]

estimate_second_type_error(df_1, df_2, 'vals', effects)

{1.03: 0.8838, 1.05: 0.7796, 1.1: 0.3828}